In [30]:
from datasets import load_dataset
import os
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric,Dataset,concatenate_datasets,set_caching_enabled, ClassLabel
import pandas as pd

import random
from IPython.display import display, HTML

import json
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2ForCTC,Wav2Vec2Processor,Trainer,TrainingArguments,Wav2Vec2FeatureExtractor

import re
set_caching_enabled(False)

import logging
import transformers
transformers.logging.get_verbosity = lambda: logging.NOTSET

transformers.logging.get_verbosity()


import soundfile as sf
import torchaudio


import IPython.display as ipd

import numpy as np
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from tqdm import tqdm
import torch

# Set environment variables
os.environ['WANDB_DISABLED '] = 'True'

import warnings
warnings.filterwarnings('ignore')

In [31]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained("prajin/wav2vec2-large-xlsr-300m-nepali")
model = Wav2Vec2ForCTC.from_pretrained("prajin/wav2vec2-large-xlsr-300m-nepali")

In [32]:
import datasets
datasets.logging.get_verbosity = lambda: logging.NOTSET

import pandas as pd

In [33]:
processed_data_dir_path = "/workspace/data_processed"

In [34]:
data_1 = '/workspace/data/external/13'

In [35]:
transcript_json = data_1 + "/" +  "data.json"
print(transcript_json)
data = pd.read_json(transcript_json)
for i in range(len(data)):
    file = data_1  + "/" + data['audioFilename'][i]
    data['audioFilename'][i] = file

/workspace/data/external/13/data.json


In [36]:
data.rename(columns = {'audioFilename':'path'}, inplace = True)
data.rename(columns = {'text':'sentence'}, inplace = True)

train = data[0:10]

len(train)

10

In [37]:
common_voice_train_1 = Dataset.from_pandas(train[:])


In [38]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = sf.read(batch["path"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch


In [39]:
common_voice_train_1 = common_voice_train_1.map(speech_file_to_array_fn, remove_columns=common_voice_train_1.column_names,num_proc=4)

In [44]:
common_voice_train_1
dataset = common_voice_train_1
dataset
audio_sample = dataset

In [42]:
# # audio_sample = dataset[3]
# audio_sample

In [54]:
audio_sample['target_text']

['लोकसभा चुनाउमा ज्येष्ठ नागरिक र सार्वजनिक  मतहरूमा मतदानको क्षेत्रमा सहायता लागि पर्वका मतदान केन्द्रमा ग्रहका निर्वाचन',
 'श्री सिंहले भन्नुभएको पूर्वोत्तर राज्यहरूलाई विभिन्न परियोजनाहरू पुनः कार्यदल गराउनका लागि दुई हजार उन्नाइस विष वित् वर्षमा जम्मा पाँच क्षेत्रको रूपमा आवन्टन दिइएको छ',
 'वनमन्त्री परिमाण',
 'स्मरणीय साहलाई मैदा प्रदान यो सटिक समय हो',
 'विवाद समाधानको निम्ति केन्द्रीय सरकारले क्षेत्रमा केन्द्रीय सुरक्षा बल तैनाथ गर्ने सिद्धान्त ग्रहण गरेको छ',
 'सुन्ने उनको समाचार राजालाई पुरा भयो',
 'भन्नुभएको छ कि दोस्रो विश्वयुद्ध जस्तै कवि नाइसिन जापानी',
 'डोडा बलबाट आउने र जाने समयको बीचमा उपयुक्त योगदान रक्षा गर्ने आदि विषयमा कुरा आरोप गर्नुभएको छ',
 'समारोहमा लोदीले पण्डित दिन दयाल पेट्रोलियम विश्वविद्यालयको उद्भावन र समाय केन्द्र लगायत परिवर्तन सिल गला केन्द्र र खेल परिसर खोला गर्नुहुनेछ',
 'यता राज्यमा अहिले बाढीको बैङ्क प्रकोप सामान्य रूपमा घटना थालेको छ भन्दै मन्त्री शेखावत भन्नुभयो कि आउँदो केही दिनमा परिस्थिति स्वाभाविक भएपछि असम सरकारले क्षतिको प्रतिवेदन']

In [46]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained("prajin/wav2vec2-large-xlsr-300m-nepali")
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}
# sorted_vocab_dict


In [47]:
model = Wav2Vec2ForCTC.from_pretrained("prajin/wav2vec2-large-xlsr-300m-nepali")
" ".join(sorted(processor.tokenizer.get_vocab()))

'[PAD] [UNK] | ँ ं ः अ आ इ ई उ ऊ ऋ ए ऐ ओ औ क ख ग घ ङ च छ ज झ ञ ट ठ ड ढ ण त थ द ध न प फ ब भ म य र ल व श ष स ह ा ि ी ु ू ृ े ै ो ौ ् ॠ'

In [53]:
import torch
for i in range(len(audio_sample)):
    inputs = processor(audio_sample[i]["speech"], sampling_rate=audio_sample[i]["sampling_rate"], return_tensors="pt")

    with torch.no_grad():
        logits = model(**inputs).logits
        
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    print(transcription[0].lower())
    

लकसभा चुनावमा जयेष्ठ नागरिक र सारैरिक कुमताहरूलाई मतदालको क्षेत्रमा सहायका लागि प्रतिका मतदान केन्द्रमा वनरो ग्रह केन्द्रसाले निर्वाचन यलेकोछ
शसिंहले भन्नुभएको पूर्वोत्तर राज्यहरूलाई विभिन्न परियोजनाहरू पुन कार्यल गराउनका लागि दुजर उन्नाइस बिस वित्त वर्षमा जम्मा पाँच स क्षरक रूपैयाँ आवन्टन दिएको छ
वनमन्त्री परिम
स्मरणय साहसलाई मै्ता प्रदानको यो सटिक समय हो
विभाद समाधानको निम्ति केन्द्रीय सरकारले क्षेत्रमा केन्द्रीय सुरक्षा ल तैनाथ गर्ने सिद्धान्त ग्रहण गरेको छ
सुन्दनको समाचार राजलाई पुरा भयो
भन्नुभएको छ कि दोस्रो विषययुद्ध जस्तै कवि नाइसिन ला पनि
दोडा दलबाट आउने र जाने समयको बीचमा उपयुक्त व्यान रक्षा गर्ने आदि विषयमा कु आरो गर्नुभएको छ
समारोमा श्रीलोदीले पण्डित दिन जाल पेट्रोलियम विश्वविद्यालयको उद्भावन र स्माय केन्द्र लगायत परिवर्तनसील गरदशला केन्द्र र खेलपरिसर खोला गर्नुहुनेछ
यता राज्यमा अहिले बारीको बैङ्क प्रकुख सामान्य रूपमा घटन थालेको छ भन्दै मन्त्री शेखावतले भन्नुभयो कि आउँदो केही दिनमा परिस्थिति स्वाभाविक भएपछि असम सरकारले क्षतिको प्रतिवेदन


In [48]:
inputs = processor(audio_sample["speech"], sampling_rate=audio_sample["sampling_rate"], return_tensors="pt")

ValueError: The model corresponding to this feature extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "Wav2Vec2ProcessorWithLM",
  "return_attention_mask": true,
  "sampling_rate": 16000
}
 was trained using a sampling rate of 16000. Please make sure that the provided `raw_speech` input was sampled with 16000 and not [16000, 16000, 16000, 16000, 16000, 16000, 16000, 16000, 16000, 16000].

In [17]:
import torch

with torch.no_grad():
    logits = model(**inputs).logits

In [18]:
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

transcription[0].lower()

'स्मरणय साहसलाई मै्ता प्रदानको यो सटिक समय हो'

# With LM decoder

In [81]:
!pip install https://github.com/kpu/kenlm/archive/master.zip pyctcdecode
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets torchaudio
import pyctcdecode

  Using cached https://github.com/kpu/kenlm/archive/master.zip
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ej4rdetd
  Running command git clone --filter=blob:none -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-ej4rdetd
  Resolved https://github.com/huggingface/transformers.git to commit f380bf2b612e6030ef8bc8904b287d274f035e29
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.17.0.dev0-py3-none-any.whl size=3598565 sha256=90adb01707b25e893407aea15adac73aab54ae831205d34f8a67ab3e0b904753
  Stored in directory: /tmp/pip-ephem-wheel-cache-20ohlboi/wheels/05/0a/97/64ae47c27ba95fae2cb5838e7b4b7247a34d4a8ba5f7092de2
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.16.0.dev0
    Uninstall

In [19]:
from transformers import Wav2Vec2ProcessorWithLM
processor = Wav2Vec2ProcessorWithLM.from_pretrained("prajin/wav2vec2-large-xlsr-300m-nepali")

In [20]:
processor.tokenizer

PreTrainedTokenizer(name_or_path='prajin/wav2vec2-large-xlsr-300m-nepali', vocab_size=62, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'pad_token': '[PAD]'})

In [21]:
" ".join(sorted(processor.tokenizer.get_vocab()))

'[PAD] [UNK] | ँ ं ः अ आ इ ई उ ऊ ऋ ए ऐ ओ औ क ख ग घ ङ च छ ज झ ञ ट ठ ड ढ ण त थ द ध न प फ ब भ म य र ल व श ष स ह ा ि ी ु ू ृ े ै ो ौ ् ॠ'

In [22]:
transcription = processor.batch_decode(logits.numpy()).text
transcription[0].lower()

'स्मरणीय साहसलाई मैता प्रधानको यो सटीक समय हो'

In [23]:
audio_sample["target_text"].lower()

'स्मरणीय साहलाई मैदा प्रदान यो सटिक समय हो'

In [28]:
predictions = ['स्मरणय साहसलाई मै्ता प्रदानको यो सटिक समय हो']
references = ['स्मरणीय साहलाई मैदा प्रदान यो सटिक समय हो']

In [29]:
wer = datasets.load_metric("wer")
wer_score = wer.compute(predictions=predictions, references=references)
print(wer_score)


0.5


In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))